# Folder-wise Fast Microscopy Picture Analysis
The following code opens a folder of the SlideScanner Microscope and automatically sets ROIs to the darkest and brightest portions of the picture. Consequently, it removes the background through the ROIs at the darkest positions and calculates the mean fluorescence at the brighter spots. 

In [1]:
## Import necessary libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tifffile as tiff
import os
import pandas as pd
import json
from datetime import datetime
from skimage import measure, draw
from PIL import Image
from tqdm import tqdm
import h5py

import gc
import concurrent.futures as cf
import multiprocessing
import threading

from AutoImgUtils import * 

In [2]:
## Set up Matplotlib
matplotlib.use('Agg')  # Use the Agg backend for non-interactive plotting
plt_lock = threading.Lock()

# Function Definitions

In [3]:
def convert_to_serializable(obj):
    """
    Convert NumPy types to native Python types for JSON serialization.
    Handles nested dictionaries and lists.
    """
    if isinstance(obj, (np.integer, np.int64, np.uint64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    elif isinstance(obj, (np.bool_,)):
        return bool(obj)
    elif obj is None or isinstance(obj, (str, bool, int, float)):
        return obj
    else:
        # Try to convert other types to string or their representation
        try:
            return str(obj)
        except:
            return repr(obj)

In [4]:
def calculate_optimal_radius(reference_image_path, config):
    """
    Calculate the optimal ROI radius from a reference image.
    
    Parameters:
        reference_image_path (str): Path to the reference image
        config (dict): Configuration parameters
        
    Returns:
        int: The calculated optimal radius
    """
    print(f"Calculating optimal radius from reference image: {os.path.basename(reference_image_path)}")
    
    # Extract parameters with defaults
    lower_thresh_chan = config.get('lower_thresh_factor', [2, 3, 2, 2])
    upper_thresh = config.get('upper_thresh', 60000)
    background_threshold = config.get('background_threshold', None)
    mask_channel = config.get('mask_channel', 1) - 1
    channel_of_interest = config.get('channel_of_interest', 1) - 1
    single_ch_background = config.get('single_ch_background', True)
    
    # Load the image
    image = tiff.imread(reference_image_path)
    image = np.moveaxis(image, 0, -1)
    
    # Subtract background
    if single_ch_background:
        background_values, mean_background_value, background_subtracted_image = bg_substraction_ROI_single_ch(
            image, background_threshold, channel_of_interest, display_rois=False)
    else:
        background_values, mean_background_value, background_subtracted_image = bg_substraction_ROI(
            image, background_threshold, display_rois=False)
    
    # Find regions in mask channel
    channel_thresh = 2 * np.std(background_subtracted_image[:, :, mask_channel]) + np.mean(background_subtracted_image[:, :, mask_channel])
    print(f'Channel threshold for mask channel {mask_channel+1}: {channel_thresh}')
    thresh = (background_subtracted_image[:, :, mask_channel] > channel_thresh) & (background_subtracted_image[:, :, mask_channel] < upper_thresh)
    labels = measure.label(thresh)
    mask_props = measure.regionprops(labels)
    
    # Find regions in channel of interest
    channel_thresh_interest = lower_thresh_chan[channel_of_interest] * np.std(background_subtracted_image[:, :, channel_of_interest]) + np.mean(background_subtracted_image[:, :, channel_of_interest])
    print(f'Channel threshold for channel of interest {channel_of_interest+1}: {channel_thresh_interest}')
    thresh_interest = (background_subtracted_image[:, :, channel_of_interest] > channel_thresh_interest) & (background_subtracted_image[:, :, channel_of_interest] < upper_thresh)
    labels_interest = measure.label(thresh_interest)
    props_interest = measure.regionprops(labels_interest)
    
    # Match regions and calculate radius
    matched_radii = []
    
    for mask_prop in mask_props:
        mask_x, mask_y = int(mask_prop.centroid[1]), int(mask_prop.centroid[0])
        
        for interest_prop in props_interest:
            interest_x, interest_y = int(interest_prop.centroid[1]), int(interest_prop.centroid[0])
            
            distance = np.sqrt((mask_x - interest_x)**2 + (mask_y - interest_y)**2)
            max_distance = np.sqrt(mask_prop.area) / 2
            
            if distance <= max_distance:
                interest_radius = np.sqrt(interest_prop.area / np.pi)
                matched_radii.append(interest_radius)
                break
    
    if len(matched_radii) > 0:
        mean_radius = np.mean(matched_radii)
        std_radius = np.std(matched_radii)
        radius_factor = int(mean_radius)
        
        print(f'Mean radius from matched regions: {mean_radius:.2f} ± {std_radius:.2f} pixels')
        print(f'Found {len(matched_radii)} matching regions between mask and channel of interest')
        print(f'Using radius_factor = {radius_factor} for all subsequent processing')
    else:
        # Fall back to using the mask channel
        areas = [prop.area for prop in mask_props]
        radii = [np.sqrt(area/np.pi) for area in areas]
        mean_radius = np.mean(radii)
        radius_factor = int(mean_radius)
        
        print(f'No matching regions found. Using mask channel. Mean radius: {mean_radius:.2f} pixels')
        print(f'Using radius_factor = {radius_factor} for all subsequent processing')
    
    # Save histogram of matched radii
    plt.hist(matched_radii, bins=40, edgecolor='black')
    plt.title('Histogram of Matched Radii')
    plt.xlabel('Radius (pixels)')
    plt.ylabel('Frequency')
    histogram_path = os.path.join(os.path.dirname(reference_image_path), 'matched_radii_histogram.png')
    plt.savefig(histogram_path)
    plt.close()
    print(f'Histogram of matched radii saved to {histogram_path}')
    

    # Force garbage collection to free memory 
    gc.collect()
    
    return radius_factor

In [5]:
def process_images(stacked_image_path, config):
    '''
    # Function to process a single set of 4-channel images and return analysis results
    
    Parameters:
        stacked_image_path (string): Path to the multi-channel TIFF image
        config (dict): Configuration parameters including:
            - lower_thresh_factor: List of threshold factors for each channel
            - upper_thresh: Upper threshold to avoid very bright spots
            - background_threshold: Threshold for background values
            - radius_factor: Radius for circular ROIs
            - channel_of_interest: Primary channel to analyze (1-indexed)
            - mask_channel: Channel to use for ROI detection (1-indexed)
            - single_ch_background: Whether to calculate background per channel
            
    Returns:
        dict: A structured dictionary containing all analysis results:
            - image_info: Basic image metadata
            - summary: Aggregated statistics across all cells
            - background: Background measurement values
            - cell_data: Individual measurements for all detected cells
            - roi_data: Legacy ROI information for backward compatibility
    '''
    
    # Extract parameters with defaults
    lower_thresh_chan = config.get('lower_thresh_factor', [2, 3, 2, 2])
    upper_thresh = config.get('upper_thresh', 60000)
    background_threshold = config.get('background_threshold', None)
    radius_factor = config.get('radius_factor', 10)
    mask_channel = config.get('mask_channel', 1)
    channel_of_interest = config.get('channel_of_interest', 1)
    single_ch_background = config.get('single_ch_background', True)

    channel_of_interest -= 1
    mask_channel -= 1
    
    image = tiff.imread(stacked_image_path)
    image = np.moveaxis(image, 0, -1)
    base_name = os.path.splitext(stacked_image_path)[0]

    n_channels = image.shape[2]

    # Initialize dictionaries to store the mean fluorescence and background values 
    mean_fluorescence = {f'Channel {i+1}': [] for i in range(n_channels)}
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    positive_results = {f'Channel {i+1}': [] for i in range(n_channels)}
    corrected_total_fluorescence = {f'Channel {i+1}': [] for i in range(n_channels)}
    
    # NEW: Create dictionary to store individual cell data with positive/negative flags
    cell_data = {f'Channel {i+1}': [] for i in range(n_channels)}

    # Subtract background from the image and display the background ROIs on the channel of interest
    if single_ch_background:
        background_values, mean_background_value, background_subtracted_image = bg_substraction_ROI_single_ch(image, background_threshold,channel_of_interest, display_rois=False)
    else:
        background_values, mean_background_value, background_subtracted_image = bg_substraction_ROI(image, background_threshold, display_rois=False)

    # Display the histogram of the background subtracted image
    # Use explicit figure creation and management
    fig = plt.figure(figsize=(n_channels*4, 10))
    axs = fig.subplots(1, n_channels)

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.hist(image[:, :, channel_index].ravel(), bins=256, color='gray', alpha=0.75)
        ax.set_title(f"Histogram for Channel {channel_index+1}")
        ax.set_xlabel("Pixel intensity")
        ax.set_ylabel("Frequency")
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.axvline(mean_background_value[channel_index], color='r', linestyle='dashed', linewidth=1)
        ax.axvline(lower_thresh_chan[channel_index] * np.std(image[:, :, channel_index]) + np.mean(image[:, :, channel_index]), color='g', linestyle='dashed', linewidth=1)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
    
    output_path = base_name + "_0_histogram.png"
    with plt_lock:
        fig.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

    # Apply thresholds to find maximum values in the background-subtracted depending on channel of interest, avoiding very bright spots
    channel_thresh = 2 * np.std(background_subtracted_image[:, :, mask_channel]) + np.mean(background_subtracted_image[:, :, mask_channel])
    print(f'Channel threshold for mask channel {mask_channel+1}: {channel_thresh}')
    thresh = (background_subtracted_image[:, :, mask_channel] > channel_thresh) & (background_subtracted_image[:, :, mask_channel] < upper_thresh)

    # Label the thresholded regions and return the number of cells
    labels = measure.label(thresh)
    props = measure.regionprops(labels)
    
    # Create circular ROIs around detected points
    rois = []

    for prop in props:
        y, x = prop.centroid
        radius = radius_factor 
        rois.append((int(x), int(y), int(radius)))

    # Calculate mean fluorescence values for each channel and check if the signal is present
    for channel_index in range(n_channels):
        channel = background_subtracted_image[:, :, channel_index]
        channel_thresh = 2 * lower_thresh_chan[channel_index] * np.std(channel) + np.mean(channel)

        # Store each ROI index for reference
        roi_index = 0
        
        for roi in rois:
            x, y, radius = roi
            rr, cc = draw.disk((y, x), radius, shape=channel.shape)
            roi_area = channel[rr, cc]
            mean_value = np.mean(roi_area)
            integrated_density = np.sum(roi_area)
            
            # Determine if the cell is positive based on the threshold
            is_positive = mean_value > channel_thresh
            
            # Store ALL cell data regardless of positive/negative status
            cell_data[f'Channel {channel_index+1}'].append({
                'roi_index': roi_index,
                'position': (x, y, radius),
                'mean_fluorescence': float(mean_value),  # Convert numpy types to native Python for serialization
                'integrated_density': float(integrated_density),
                'is_positive': bool(is_positive)
            })
            
            # For backward compatibility, maintain previous data structures
            if is_positive:
                positive_results[f'Channel {channel_index+1}'].append((x, y, radius))
                mean_fluorescence[f'Channel {channel_index+1}'].append(mean_value)
                corrected_total_fluorescence[f'Channel {channel_index+1}'].append(integrated_density)
            else:
                positive_results[f'Channel {channel_index+1}'].append(None)
                corrected_total_fluorescence[f'Channel {channel_index+1}'].append(None)
            
            roi_index += 1

    # fig = plt.figure(figsize=(n_channels * 5, 20))
    # axs = fig.subplots(1, n_channels)

    # axs[0].imshow(normalize(image[:,:,0]), cmap='gray')
    # axs[1].imshow(normalize(image[:,:,1]), cmap='gray')
    # axs[2].imshow(normalize(image[:,:,2]), cmap='gray')
    # axs[3].imshow(normalize(image[:,:,3]), cmap='gray')

    # for ax, channel_index in zip(axs, range(n_channels)):
    #     ax.set_title(f'Channel {channel_index+1} (Raw)')

    # # Hide x labels and tick labels for top plots and y ticks for right plots.
    # for ax in axs.flat:
    #     ax.label_outer()
    #     ax.axis('off')

    # output_path = base_name + "_1_Originals.png"
    # with plt_lock:
    #     fig.savefig(output_path, bbox_inches='tight', pad_inches=0)
    #     plt.close(fig)

    # plt.show()

    # Delete original image to free memory
    del image

    # Display positive ROIs for each channel
    fig = plt.figure(figsize=(n_channels * 5, 20))
    ax = fig.subplots(1, n_channels)

    for channel_index in range(n_channels):
        channel_image = np.stack([normalize(background_subtracted_image[:, :, channel_index])]*3, axis=-1)  # Convert to RGB

        for roi in positive_results[f'Channel {channel_index+1}']:
            if roi is not None:
                x, y, radius = roi
                rr, cc = draw.disk((y, x), radius, shape=channel_image.shape)
                channel_image[rr, cc] = [0, 1, 0]  # Green for positive ROIs

        ax[channel_index].imshow(channel_image)
        ax[channel_index].set_title(f'Positive ROIs on Channel {channel_index + 1}')
        ax[channel_index].axis('off')

    output_path = base_name + "_2_ROIs.png"
    with plt_lock:
        fig.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
    
    # Force garbage collection to free memory
    plt.close('all') 
    gc.collect()

    # Build a unified results dictionary
    results = {
        'image_info': {
            'path': stacked_image_path,
            'filename': os.path.basename(stacked_image_path),
            'channels': n_channels,
            'shape': image.shape if 'image' in locals() else None
        },
        'summary': {
            'mean_fluorescence': {ch: np.mean(values) if values else None for ch, values in mean_fluorescence.items()},
            'positive_count': {ch: sum(x is not None for x in positive_results[ch]) for ch in positive_results},
            'negative_count': {ch: sum(x is None for x in positive_results[ch]) for ch in positive_results},
            'total_cells': len(rois)
        },
        'background': {
            'values': background_values,
            'mean_value': mean_background_value
        },
        'cell_data': cell_data,
        'roi_data': {
            'positive_results': positive_results,
            'corrected_total_fluorescence': corrected_total_fluorescence
        }
    }

    return results

In [6]:
def process_file(filepath, config):
    """Process a single file with the given parameters and return results."""
    
    try:
        print(f"Processing {os.path.basename(filepath)}...")
        
        # Create a thread-local figure manager
        with plt.rc_context():
            # Force matplotlib to create new figures for this thread
            plt.close('all')
            plt.ioff()  # Turn off interactive mode
            
            # Process the image and get results dictionary
            results = process_images(filepath, config)
            
            # Generate result dictionary for CSV
            csv_result = {'Base Name': filepath}
            
            # Add summary metrics to CSV results
            for channel, mean_value in results['summary']['mean_fluorescence'].items():
                csv_result[f'{channel} Fluorescence mean value'] = mean_value
                csv_result[f'{channel} Mean Background'] = np.mean(results['background']['values'][channel]) if results['background']['values'][channel] else None
                csv_result[f'{channel} Positive Results'] = results['summary']['positive_count'][channel]
                csv_result[f'{channel} Negative Results'] = results['summary']['negative_count'][channel]
                
                # Get corrected total fluorescence values (non-None values)
                ctf_values = [x for x in results['roi_data']['corrected_total_fluorescence'][channel] if x is not None]
                csv_result[f'{channel} Corrected Total Fluorescence'] = np.mean(ctf_values) if ctf_values else None
            
            # Make sure all figures are closed
            plt.close('all')
            
            # Save individual cell data to a JSON file
            base_name = os.path.splitext(filepath)[0]
            cell_data_path = f"{base_name}_cell_data.json"
            with open(cell_data_path, 'w') as f:
                # Extract just the cell data portion for the individual file
                json.dump(results['cell_data'], f, indent=2)
            
        # Explicit garbage collection
        gc.collect()
        return csv_result, results
    
    except Exception as e:
        print(f"Error processing {filepath}: {str(e)}")
        plt.close('all')  # Make sure to close any open figures on error
        return None, None

In [7]:
def show_tiff_image(image_path):
    """Display a tiff image using matplotlib."""
    image = tiff.imread(image_path)
    image = np.moveaxis(image, 0, -1)

    n_channels = image.shape[2]
    
    fig, axs = plt.subplots(1, n_channels , figsize=(n_channels *5,20))

    axs[0].imshow(normalize(image[:,:,0]), cmap='gray')
    axs[1].imshow(normalize(image[:,:,1]), cmap='gray')
    axs[2].imshow(normalize(image[:,:,2]), cmap='gray')
    axs[3].imshow(normalize(image[:,:,3]), cmap='gray')

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.set_title(f'Channel {channel_index+1} (Raw)')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
        ax.axis('off')
    
    plt.show()

In [8]:
def process_folder(folder_path, config):
    """Process all files in the given folder with the provided configuration."""
    
    # Get all TIFF files in the folder
    tiff_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                 if f.lower().endswith(('.tif', '.tiff'))]
    
    if not tiff_files:
        print(f"No TIFF files found in {folder_path}")
        return None
    
    print(f"Found {len(tiff_files)} files to process")

    # Process all files using ThreadPoolExecutor
    csv_results = []
    
    # Create a compact summary structure for metadata
    experiment_metadata = {
        'experiment_folder': folder_path,
        'processed_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'configuration': convert_to_serializable(config),
        'file_count': len(tiff_files)
    }
    
    max_workers = max(4, os.cpu_count() // 2)  # Limit workers to avoid memory issues
    print(f"Processing files with {max_workers} parallel workers")
    
    # Create results directory if it doesn't exist
    results_dir = os.path.join(folder_path, "results")
    os.makedirs(results_dir, exist_ok=True)
    
    # HDF5 file path
    hdf5_path = os.path.join(results_dir, "experiment_data.h5")
    
    # Process images and collect results
    image_results = {}
    
    with cf.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_file = {executor.submit(process_file, f, config): f for f in tiff_files}
        
        # Process results as they complete with progress bar
        with tqdm(total=len(tiff_files), desc="Processing Images", unit="file") as progress_bar:
            for future in cf.as_completed(future_to_file):
                file_path = future_to_file[future]
                file_name = os.path.basename(file_path)
                
                try:
                    csv_result, full_results = future.result()
                    if csv_result:
                        csv_results.append(csv_result)
                        image_results[file_name] = {
                            'summary': convert_to_serializable(full_results['summary']),
                            'cell_count': full_results['summary']['total_cells'],
                            'individual_data_file': os.path.basename(os.path.splitext(file_path)[0]) + "_cell_data.json"
                        }
                        
                except Exception as e:
                    print(f"Error processing {file_name}: {str(e)}")
                
                # Update progress bar
                progress_bar.update(1)
    
    if not csv_results:
        print("No files were successfully processed.")
        return None
    
    # Create and save HDF5 file with experiment data
    print(f"Saving experiment data to HDF5...")
    with h5py.File(hdf5_path, 'w') as f:
        # Store metadata as attributes in root group
        for key, value in experiment_metadata.items():
            if key == 'configuration':
                # Store configuration as JSON string
                f.attrs[key] = json.dumps(value)
            else:
                f.attrs[key] = value
            
        # Create images group
        images_group = f.create_group('images')
        
        # Store data for each image
        for img_name, img_data in image_results.items():
            img_group = images_group.create_group(img_name.replace('.tif', '').replace('.tiff', ''))
            
            # Store image summary
            summary_group = img_group.create_group('summary')
            
            # Store mean fluorescence for each channel
            for channel, value in img_data['summary']['mean_fluorescence'].items():
                if value is not None:
                    summary_group.attrs[f'mean_fluorescence_{channel}'] = value
            
            # Store counts
            for count_type in ['positive_count', 'negative_count']:
                count_group = summary_group.create_group(count_type)
                for channel, value in img_data['summary'][count_type].items():
                    count_group.attrs[channel] = value
            
            # Store other metadata
            img_group.attrs['cell_count'] = img_data['cell_count']
            img_group.attrs['individual_data_file'] = img_data['individual_data_file']
    
    print(f"Experiment data saved to {hdf5_path}")
    
    # Also save a small JSON version of the metadata for easy inspection
    metadata_json_path = os.path.join(results_dir, "experiment_metadata.json")
    with open(metadata_json_path, 'w') as f:
        json.dump({
            'metadata': experiment_metadata,
            'image_count': len(image_results),
            'hdf5_file': os.path.basename(hdf5_path)
        }, f, indent=2)
    
    print(f"Experiment metadata saved to {metadata_json_path}")
    
    # Create a DataFrame from the results and save to CSV
    df = pd.DataFrame(csv_results)
    csv_path = os.path.join(results_dir, "results.csv")
    df.to_csv(csv_path, index=False)
    
    print(f"Results saved to {csv_path}")
    return df

In [9]:
def process_experiments_batch(main_directory, config):
    """
    Process multiple experiment folders contained within a main directory.
    Always processes all folders, overwriting any existing results.
    
    Parameters:
        main_directory (str): Path to the directory containing multiple experiment folders
        config (dict): Configuration dictionary for processing
        
    Returns:
        dict: Summary of processing results for all experiments
    """
    # Get all subdirectories in the main directory
    experiment_folders = [os.path.join(main_directory, d) for d in os.listdir(main_directory) 
                         if os.path.isdir(os.path.join(main_directory, d))]
    
    if not experiment_folders:
        print(f"No experiment folders found in {main_directory}")
        return None
    
    total_experiments = len(experiment_folders)
    print(f"Found {total_experiments} experiment folders to process")
    
    # Create a summary dictionary
    summary = {
        'main_directory': main_directory,
        'start_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'config': config,
        'experiments': {},
        'total_experiments': total_experiments,
        'successful_experiments': 0,
        'failed_experiments': 0
    }
    
    # Process each experiment folder sequentially
    for i, folder in enumerate(experiment_folders, 1):
        folder_name = os.path.basename(folder)
        
        # Check for existing results and warn user
        results_path = os.path.join(folder, "results", "experiment_data.json")
        if os.path.exists(results_path):
            print(f"\n[{i}/{total_experiments}] Processing {folder_name} (overwriting existing results)")
        else:
            print(f"\n[{i}/{total_experiments}] Processing experiment: {folder_name}")
        
        try:
            # Use the provided configuration without modification
            folder_config = config.copy()
            
            # Process the folder
            print(f"Using radius factor: {folder_config['radius_factor']}")
            df = process_folder(folder, folder_config)
            
            if df is not None:
                summary['experiments'][folder_name] = {
                    'status': 'success',
                    'files_processed': len(df),
                    'config': folder_config
                }
                summary['successful_experiments'] += 1
            else:
                summary['experiments'][folder_name] = {
                    'status': 'failed',
                    'reason': 'No files were successfully processed',
                    'config': folder_config
                }
                summary['failed_experiments'] += 1
                
        except Exception as e:
            print(f"Error processing folder {folder_name}: {str(e)}")
            import traceback
            traceback.print_exc()  # Print the full stack trace for better debugging
            summary['experiments'][folder_name] = {
                'status': 'failed',
                'reason': str(e),
                'config': config
            }
            summary['failed_experiments'] += 1
    
    # Add end time to summary
    summary['end_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    start_time = datetime.strptime(summary['start_time'], "%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(summary['end_time'], "%Y-%m-%d %H:%M:%S")
    summary['duration_seconds'] = (end_time - start_time).total_seconds()
    
    # Save summary to JSON
    summary_path = os.path.join(main_directory, "batch_processing_summary.json")
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"\nBatch processing complete:")
    print(f"- Successful: {summary['successful_experiments']}")
    print(f"- Failed: {summary['failed_experiments']}")
    print(f"Total time: {summary['duration_seconds'] / 60:.2f} minutes")
    print(f"Summary saved to {summary_path}")
    
    return summary

# Main Workflow

In [10]:
# Alternative method to select the folder (open main folder with all subfolders)
main_folder_path = select_folder()

In [ ]:
# Select refrerence image for radius calculation
reference_image_path = select_file()

In [11]:
# Define processing configuration
config = {
    'lower_thresh_factor': [2, 2, 2, 2],
    'upper_thresh': 50000,
    'background_threshold': None,
    'radius_factor': 17,
    'mask_channel': 1,
    'channel_of_interest': 4,
    'single_ch_background': True
}

if config['radius_factor'] is None:
    radius_factor = calculate_optimal_radius(reference_image_path, config)
    config['radius_factor'] = radius_factor
else:  
    print(f'Using predefined radius factor: {config["radius_factor"]}')

Using predefined radius factor: 17


In [ ]:
# Process an individual selected folder
df = process_folder(main_folder_path, config)
print("processing complete")

In [ ]:
# Process an experiment batch in the selected main folder
main_directory = main_folder_path

if main_directory:
    
    print("Starting batch processing with the following configuration:")
    print(f"- Radius factor: {config['radius_factor']}")
    print(f"- Mask channel: {config['mask_channel']}")
    print(f"- Channel of interest: {config['channel_of_interest']}")
    print(f"- Lower threshold factors: {config['lower_thresh_factor']}")
    
    # Process all experiment folders
    batch_summary = process_experiments_batch(main_directory, config)
    print("Batch processing complete.")
else:
    print("No directory selected. Processing cancelled.")

Starting batch processing with the following configuration:
- Radius factor: 17
- Mask channel: 1
- Channel of interest: 4
- Lower threshold factors: [2, 2, 2, 2]
Found 3 experiment folders to process

[1/3] Processing EXP01 (overwriting existing results)
Using radius factor: 17
Found 2 files to process
Processing files with 16 parallel workers
Processing EXP10_3_1_1_NB_BIC_5CO_scFL_NO_Doxy_1-Scene-1-ScanRegion0-OME.ome.tiff...
Processing EXP10_3_1_1_NB_BIC_5CO_scFL_NO_Doxy_1-Scene-2-ScanRegion1-OME.ome.tiff...


Processing Images:   0%|          | 0/2 [00:00<?, ?file/s]

Background threshold for channel 4: 315.8717614040461
Background threshold for channel 4: 322.3237002752985
